In [1]:
import geopandas as gp
import csv
import pandas as pd
from shapely.geometry import LineString
import mysql.connector
from shapely.geometry import Polygon, mapping
import datetime
import json 

In [113]:
file = "./KY/KY.geojson"    

precincts = gp.read_file(file)
precincts.head()

,area,id,type,centerX,centerY,canonical,neighbors,geometry
0,0.002012,21179D105,ORIGINAL,-85.47519600000001,37.849999,KY Nelson D105,21179D103,"MULTIPOLYGON (((-85.47258 37.82863, -85.48071 ..."
1,0.005112,21149C102,ORIGINAL,-87.10202099999999,37.532737,KY McLean C102,None,"MULTIPOLYGON (((-87.15110 37.49865, -87.15485 ..."
2,0.020443,21147F106,ORIGINAL,-84.6542855,36.6619115,KY McCreary F106,None,"MULTIPOLYGON (((-84.60974 36.72119, -84.60625 ..."
3,0.006973,21147A104,ORIGINAL,-84.4995435,36.6219995,KY McCreary A104,None,"MULTIPOLYGON (((-84.60519 36.59878, -84.60444 ..."
4,0.002376,21147F109,ORIGINAL,-84.51780650000001,36.655203,KY McCreary F109,None,"MULTIPOLYGON (((-84.50946 36.62620, -84.50964 ..."


In [ ]:
reader = csv.reader(open('./KY_county.csv', 'r'))
d = {}
for row in reader:
    #id = row['County FIPS']
    key = row[0]
    value = row[1]
    d[key] = value
#print (d)

In [ ]:
#Rename columns
#df = df.drop (columns = ['state_coun', 'democrat_p', 'republic_1', 'male_perce', 'republican',
                         #'democrat_p','republic_1','name', 'population', 'democrats', 'SUM',
                       # 'cartodb_id', 'other', 'other_perc', 'male', 'male_perce', 'female',
                        # 'female_per', 'total_regi', 'registered', 'id', 'data', 'dem_rep_di'])

df.rename(columns = {"PRECINCT_C":"id"}, inplace = True)
df.rename(columns = {"GEOID10":"canonical"}, inplace = True)
#df.rename(columns = {"NEIGHBORS":"neighbors"}, inplace = True)
#df.rename(columns = {"INTPTLON10":"centerY"}, inplace = True)

#Add type columns
df.insert(2, 'type', 'ORIGINAL')
df.insert(3, 'centerX', None)
df.insert(4, 'centerY', None)
#df.insert(5, 'area', None)

#df = df.drop ()
df.head()

In [ ]:
#Canonical name
state = ''
for i, row in df.iterrows():   
    id = str(df.at[i, 'id'])
    countyId = '21' + str(df.at[i, 'COUNTYFP10'])
    #print (countyId)
    canonicalId = state + ' ' + d[countyId] + ' ' + id
    df.at[i, 'canonical'] = canonicalId
    df.at[i, 'id'] = '42' + str(df.at[i, 'COUNTYFP10'])
    #print (canonicalId)


#Drop column

df.head()
#df.head()

In [ ]:
#Replace multipoly with polygon
arr_polygon = [0] * len(df)
arr_polygon = gp.GeoDataFrame (arr_polygon, columns = ['geometry'])
for index, row in df.iterrows():
    #Progress
    if index % 1000 == 0:
        print (index, datetime.datetime.now())
    polygon = row.geometry[0]
    arr_polygon.loc[index] = polygon
    
df['geometry'] = arr_polygon
df.head()    

In [ ]:
#ADD area and centers
for i, row in df.iterrows():
    #df.at[i, 'area'] = row.geometry.area
    bounds = row.geometry.bounds
    centerX = (bounds[0] + bounds[2]) / 2
    centerY = (bounds[1] + bounds[3]) / 2
    df.at[i, 'centerX'] = centerX
    df.at[i, 'centerY'] = centerY
    

In [ ]:
df = df.drop(columns = ['COUNTYFP10'])
df.head()  

In [ ]:
df.to_file("./KY/KY_precinct_boundary_neighbors_high_precision.geojson", driver='GeoJSON')

In [ ]:
### GENERATE NEIGHBORS
start_time = time.time()
file = "./KY/KY.geojson"  
df = gp.read_file(file) # open file

#df.rename(columns = {"vtd":"GEOID10"}, inplace = True)
#df["neighbors"] = None  # add NEIGHBORS column

for index, row in df.iterrows():
    #what complete
    if index % 1000 == 0:
        print (datetime.datetime.now().strftime("%H:%M:%S"), index)
    # get 'not disjoint' countries
    neighbors = df[~df.geometry.disjoint(row.geometry)].id.tolist()
    # remove own name from the list
    neighbors = [ name for name in neighbors if row.id != name ]
    # add names of neighbors as NEIGHBORS value
    df.at[index, 'neighbors'] = ", ".join(neighbors)

# save GeoDataFrame as a new file
df.to_file("./KY/KY_neighbors.geojson", driver='GeoJSON')
print ("Finished in ", time.time() - start_time)

In [ ]:
### OVERLAP 

file = "../data/new/KY_precinct_boundary.geojson"
data_temp = gp.read_file(file)

#pltdata_temp=gpd.read_file('data/temp.shp')
#data_temp.head()

In [ ]:
def create_neighbors_df(neighbors, df):
    neighbors = neighbors.split(',')
    neighbors_df = df[df['id'].isin(neighbors)][['id', 'geometry']]
    return neighbors_df   

In [ ]:
file = "./KY/KY_neighbors.geojson"  
precincts = gp.read_file(file) # open file
precincts = precincts.drop(columns=['area', 'type', 'centerX', 'centerY', 'canonical'])
precincts.head()

In [ ]:
AREA_THRESHOLD = 1e-4
intersections = gp.GeoDataFrame(crs = precincts.crs)

for index, row in precincts.iterrows():
    #For progress
    if index % 1000 == 0:
        print (datetime.datetime.now().strftime("%H:%M:%S"), index)
    neighbors = create_neighbors_df(row['neighbors'], precincts)
    #print(neighbors)
    # check if intersection occured
    overlaps = neighbors[neighbors.geometry.overlaps(row.geometry)]['id'].tolist()
    if len(overlaps) > 0:
        # compare the area with threshold
        for y in overlaps:
            temp_area = gp.overlay(precincts.loc[precincts.id == y,], precincts.loc[precincts.id == row.id], how='intersection')
            temp_area = temp_area.loc[temp_area.geometry.area >= AREA_THRESHOLD]
            
            if temp_area.shape[0] > 0:
                intersections = gp.GeoDataFrame(pd.concat([temp_area, intersections], ignore_index=True),crs = precincts.crs)
                intersections['sorted'] = intersections.apply(lambda y: sorted([y['id_1'],y['id_2']]), axis=1)
                intersections['sorted'] = intersections.sorted.apply(lambda y: ''.join(y))
                intersections = intersections.drop_duplicates('sorted')
                intersections = intersections.reset_index()[['id_1','id_2','geometry']]
                intersections = gp.GeoDataFrame(pd.concat([temp_area, intersections],ignore_index=True),crs=precincts.crs)

In [ ]:
intersections.shape

In [ ]:
min_area = precincts.at[0, 'area']
max_area = precincts.at[0, 'area']
print(max_area, min_area)
for index, row in precincts.iterrows():
    if min_area > row['area']:
        min_area = row['area']
    if max_area < row['area']:
        max_area = row['area']
        
print(max_area, min_area)

In [ ]:
### GAP
data_temp_diss = data_temp[['geometry', 'type']]
data_temp_diss = data_temp.dissolve(by='type')
data_temp_diss = df.drop (columns = ['type', 'id', 'canonical', 'centerX', 'centerY', 'area'])

data_temp_diss.head()


In [ ]:
interior = data_temp_diss.interiors.values.tolist()

#print (interior)

gap_list = []
for i in interior:
    if i == []:
        continue
    #print(i[0].coords[:])    
    gap_list.append(Polygon(i[0]))
    
data_gaps = gp.GeoDataFrame(geometry = gap_list,crs=data_temp.crs)
data_gaps['feature_touches'] = data_gaps.geometry.apply(lambda y: data_temp.loc[data_temp.touches(y)]['id'].tolist())
data_gaps.tail()

In [3]:
## MAP COVERAGE ERROR

file_states = "./States/states_boundary.geojson"
state_boundary = gp.read_file(file_states)
state_boundary.head()

,id,centerX,centerY,area,geometry
0,PA,-77.6048035,41.1179355,12.7471968513065,"MULTIPOLYGON (((-80.51935 41.84956, -80.51922 ..."
1,KY,-85.76960750000001,37.82234,10.668081506716,"MULTIPOLYGON (((-88.06133 37.50533, -88.06368 ..."
2,NY,-75.77002999999999,42.7460485,15.58035125575651,"MULTIPOLYGON (((-79.31214 42.68680, -79.14347 ..."


In [4]:
file_precincts = "./KY/KY.geojson"
precinct_boundary = gp.read_file(file_precincts)
precinct_boundary.head()

,area,id,type,centerX,centerY,canonical,neighbors,geometry
0,0.002012,21179D105,ORIGINAL,-85.47519600000001,37.849999,KY Nelson D105,21179D103,"MULTIPOLYGON (((-85.47258 37.82863, -85.48071 ..."
1,0.005112,21149C102,ORIGINAL,-87.10202099999999,37.532737,KY McLean C102,None,"MULTIPOLYGON (((-87.15110 37.49865, -87.15485 ..."
2,0.020443,21147F106,ORIGINAL,-84.6542855,36.6619115,KY McCreary F106,None,"MULTIPOLYGON (((-84.60974 36.72119, -84.60625 ..."
3,0.006973,21147A104,ORIGINAL,-84.4995435,36.6219995,KY McCreary A104,None,"MULTIPOLYGON (((-84.60519 36.59878, -84.60444 ..."
4,0.002376,21147F109,ORIGINAL,-84.51780650000001,36.655203,KY McCreary F109,None,"MULTIPOLYGON (((-84.50946 36.62620, -84.50964 ..."


In [5]:
STATE = 'KY'
state = state_boundary[state_boundary['id'] == STATE]
state_area = state_boundary.at[1, 'area']
precinct_area = precinct_boundary['area'].sum()
print("Difference: ", float(state_area) - float(precinct_area))

Difference:  0.0071088365755063165


In [7]:
precinct_dissolved = precinct_boundary[['geometry', 'type']]
precinct_dissolved = precinct_dissolved.dissolve(by='type')
precinct_dissolved.head()

,geometry
type,
ORIGINAL,"MULTIPOLYGON (((-83.53183 36.66366, -83.53194 ..."


In [8]:
difference = gp.overlay(state, precinct_dissolved, how = 'difference')

In [28]:
THRESHOLD = 5e-4

add_error_querry = ("INSERT INTO mhaczynska.data_error "
               "(ERROR_TYPE, PRECINCT, CENTER_X, CENTER_Y, STATE) "
               "VALUES (%s, %s, %s, %s, %s)")

ERROR_COVERAGE_TYPE = 'COVERAGE'
STATE = 'KY'
coverage_error_data = []

for multipolygon in difference.geometry:
    for polygon in multipolygon:
        if polygon.area > THRESHOLD:
            bounds = polygon.bounds
            centerX = round((bounds[0] + bounds[2]) / 2, 6)
            centerY = round((bounds[1] + bounds[3]) / 2, 6)
            coverage_row = (ERROR_COVERAGE_TYPE, None, centerX, centerY, STATE)
            coverage_error_data.append(coverage_row)
            
cursor.executemany(add_error_querry, coverage_error_data)
mydb.commit()           
print(coverage_error_data)

[('COVERAGE', None, -87.185193, 37.563541, 'KY')]


In [39]:
difference.head()
difference.to_file("./KY/KY_difference.geojson", driver='GeoJSON')

In [2]:
### ANOMALOUS DATA
file = "./demographic_KY/precinct_demographic_KY.geojson"
demographics = gp.read_file(file)
demographics.head()

DriverError: ./demographic_KY/precinct_demographic_KY.geojson: No such file or directory

In [99]:
file = "./election/KY_president2016.json"
president = pd.read_json(file)  
file = "./election/KY_congress2016.json"
congress2016 = pd.read_json(file)
president.head()

,id,democrat,republican
0,21179D105,697.0,321.0
1,21149C102,455.0,96.0
2,21147F106,89.0,12.0
3,21147A104,284.0,30.0
4,21147F109,80.0,5.0


In [100]:
print(president.shape)
print(congress2016.shape)
print(demographics.shape)
congress2016.head()

(3583, 3)
(3583, 3)
(3583, 9)


,id,democrat,republican
0,21179D105,448,611
1,21149C102,186,391
2,21147F106,15,83
3,21147A104,46,250
4,21147F109,14,71


In [1]:
anomalous_data = []
for index, row in demographics.iterrows():    
    anomalous = False
    
    if row['total'] <= 10:
        anomalous = True       
        
    #if president.at[index, 'democrat'] == 0 and president.at[index, 'republican'] == 0:
       # anomalous = True
    
   # if congress2016.at[index, 'democrat'] == 0 and congress2016.at[index, 'republican'] == 0:        
      #  anomalous = True
        
    #if president.at[index, 'democrat'] + president.at[index, 'republican'] > row['total']:
        #anomalous = True
        
    #if anomalous:
        #anomalous_data.append(row['id'])

print(anomalous_data)


NameError: name 'demographics' is not defined

In [134]:
add_error_querry = ("INSERT INTO mhaczynska.data_error "
               "(ERROR_TYPE, PRECINCT, CENTER_X, CENTER_Y, STATE) "
               "VALUES (%s, %s, %s, %s, %s)")

STATE = 'KY'
ERROR_ANOMALOUS_TYPE = 'ANOMALOUS'
anomalous = []

for precinct_id in anomalous_data:
    centerX = precincts[precincts['id'] == precinct_id]['centerX'].values[0]
    centerY = precincts[precincts['id'] == precinct_id]['centerY'].values[0]
    anomalous_row = (ERROR_ANOMALOUS_TYPE, precinct_id, centerX, centerY, STATE)
    anomalous.append(anomalous_row)

cursor.executemany(add_error_querry, anomalous)
mydb.commit()    
print(anomalous)

[('ANOMALOUS', '21111H150', '-85.70502049999999', '38.195672', 'KY'), ('ANOMALOUS', '21111D136', '-85.60960299999999', '38.19028', 'KY'), ('ANOMALOUS', '21029X001', '-85.8367825', '37.9205795', 'KY'), ('ANOMALOUS', '21061E101', '-86.15211149999999', '37.2531295', 'KY'), ('ANOMALOUS', '21029D104', '-85.730193', '37.985405', 'KY'), ('ANOMALOUS', '21029D114', '-85.72171599999999', '37.971952', 'KY'), ('ANOMALOUS', '21227A105', '-86.441236', '36.924752', 'KY'), ('ANOMALOUS', '21221X001', '-87.7437135', '36.680863', 'KY'), ('ANOMALOUS', '21221X002', '-88.03467499999999', '36.7886345', 'KY'), ('ANOMALOUS', '21015B130', '-84.65254849999999', '38.920665', 'KY'), ('ANOMALOUS', '21073B106', '-84.780689', '38.197368', 'KY'), ('ANOMALOUS', '21111L118', '-85.685845', '38.2647095', 'KY'), ('ANOMALOUS', '21111F156', '-85.55974449999999', '38.24145350000001', 'KY'), ('ANOMALOUS', '21111C101', '-85.6906505', '38.1863175', 'KY'), ('ANOMALOUS', '21111D126', '-85.57695649999999', '38.165298', 'KY'), ('ANO

In [3]:
#POPULATE DATABASE
mydb = mysql.connector.connect(
    host="mysql3.cs.stonybrook.edu",
    user='mhaczynska',
    passwd="110670160",
    database = 'mhaczynska'    
)

cursor = mydb.cursor()
print (mydb)

In [4]:
file = "./KY/KY_neighbors.geojson" 
precincts = gp.read_file(file)
precincts.head()

,area,id,type,centerX,centerY,canonical,neighbors,geometry
0,0.002012,21179D105,ORIGINAL,-85.47519600000001,37.849999,KY Nelson D105,"21179C101, 21179D101, 21179C102, 21179B104, 21...","MULTIPOLYGON (((-85.47258 37.82863, -85.48071 ..."
1,0.005112,21149C102,ORIGINAL,-87.10202099999999,37.532737,KY McLean C102,"21183A105, 21149B101, 21059E104, 21059E103, 21...","MULTIPOLYGON (((-87.15110 37.49865, -87.15485 ..."
2,0.020443,21147F106,ORIGINAL,-84.6542855,36.6619115,KY McCreary F106,"21147A104, 21147F109, 21147F108, 21147F105, 21...","MULTIPOLYGON (((-84.60974 36.72119, -84.60625 ..."
3,0.006973,21147A104,ORIGINAL,-84.4995435,36.6219995,KY McCreary A104,"21147F106, 21147F109, 21147B101, 21147B104, 21...","MULTIPOLYGON (((-84.60519 36.59878, -84.60444 ..."
4,0.002376,21147F109,ORIGINAL,-84.51780650000001,36.655203,KY McCreary F109,"21147F106, 21147A104, 21147A101, 21147A105, 21...","MULTIPOLYGON (((-84.50946 36.62620, -84.50964 ..."


In [5]:
#ADD PRECINCTS

add_precinct_querry = ("INSERT INTO mhaczynska.precinct "
               "(GEO_ID, NAME, STATE_NAME, CENTER_X, CENTER_Y, NEIGHBORS, PRECINCT_GEOJSON) "
               "VALUES (%s, %s, %s, %s, %s, %s, %s)")

update_precinct_querry = "UPDATE mhaczynska.precinct SET CENTER_X = %s, CENTER_Y = %s WHERE GEO_ID = %s"

state = 'KY'
add_precinct = []
for index, row in precincts.iterrows():
    geometry_str = row.geometry.__geo_interface__
    geometry_str = json.dumps(geometry_str)
    precinct_data = (row['id'], row['canonical'], state, row['centerX'], row['centerY'], row['neighbors'], geometry_str)
    #update_precinct = (row['centerY'], row['centerX'], row['id'])
    #add_precinct.append(update_precinct)
    try:
        cursor.execute(add_precinct_querry, precinct_data)
        mydb.commit()
    except:
        pass
    
#cursor.executemany(update_precinct_querry, add_precinct)
#mydb.commit()

In [ ]:
precincts.shape

In [ ]:
#ADD NEIGHBORS
add_neighbor_querry = ("INSERT INTO mhaczynska.neighbors "
               "(PRECINCT, NEIGHBOR_GEOID) "
               "VALUES (%s, %s)")

neighbor_data = []
for index, row in precincts.iterrows():
    neighbors = row['neighbors']
    neighbors = neighbors.split(",")
    for neighbor in neighbors:
        neighbor_row = (row['id'], neighbor.strip())
        neighbor_data.append(neighbor_row)        
    
cursor.executemany(add_neighbor_querry, neighbor_data)
mydb.commit()

In [ ]:
#ENCLOSED PRECINCTS

add_error_querry = ("INSERT INTO mhaczynska.data_error "
               "(ERROR_TYPE, PRECINCT, CENTER_X, CENTER_Y, STATE) "
               "VALUES (%s, %s, %s, %s, %s)")

update_enclosed_error_querry = "UPDATE mhaczynska.data_error SET STATE = %s WHERE PRECINCT = %s"

ERROR_ECLOSED_TYPE = 'ENCLOSED'
ERROR_MULTIPOLYGON_TYPE = 'MULTIPOLYGON'
STATE = 'KY'
enclosed_error_data = []
multipolygon_error_data = []
for index, row in precincts.iterrows():    
    if  len(row.geometry) > 1:
        multipolygon_row = (ERROR_MULTIPOLYGON_TYPE, row['id'], row['centerX'], row['centerY'], STATE)
        multipolygon_error_data.append(multipolygon_row)
    for i, polygon in enumerate(row.geometry):
            #if i != 0:
        interior_list = list(polygon.interiors)
        if len(interior_list) > 0:
            bounds = interior_list[0].bounds
            centerX = round((bounds[0] + bounds[2]) / 2, 6)
            centerY = round((bounds[1] + bounds[3]) / 2, 6)
            enclosed_row = (ERROR_ECLOSED_TYPE, row['id'], centerX, centerY, STATE)
            enclosed_error_data.append(enclosed_row)

cursor.executemany(add_error_querry, enclosed_error_data)
mydb.commit()
cursor.executemany(add_error_querry, multipolygon_error_data)
mydb.commit()

In [ ]:
## OVERLAP 
add_error_querry = ("INSERT INTO mhaczynska.data_error "
               "(ERROR_TYPE, PRECINCT, CENTER_X, CENTER_Y, STATE) "
               "VALUES (%s, %s, %s, %s, %s)")

file = './KY/ky_points_intersection.geojson'
intersections = gp.read_file(file)
intersections_data = []
ERROR_INTERSECTIONS_TYPE = 'OVERLAP'
STATE = 'KY'
for index, row in intersections.iterrows():
    point = list(row.geometry.coords)
    intersection_row = (ERROR_INTERSECTIONS_TYPE, row['vtd'], point[0][0], point[0][1], STATE)
    intersections_data.append(intersection_row)

cursor.executemany(add_error_querry, intersections_data)
mydb.commit()    
print(intersections_data)

In [6]:
cursor.close()
mydb.close()